# Intro to Scientific Programming
By G Hosseinzadeh 2025 Apr 15

Let's imagine you want to generate synthetic spectra of different types of stars. The figure below shows approximate temperature and radius ranges for each type. In reality they have some absorption lines (shown at left), but for now let's approximate them as perfect blackbody emitters.

![main sequence stars](https://upload.wikimedia.org/wikipedia/commons/3/37/Stellar_Classification_Chart.png)  
Credit: Pablo Carlos Budassi (Wikimedia.org)

The Planck function gives the spectral radiance $(B_\lambda \equiv \frac{dE}{dt\,d\lambda\,dA\,d\Omega})$ of the stellar photosphere for a given temperature:
$$B_\lambda = \frac{2 h c^2}{\lambda^5} \frac{1}{\exp\left(\frac{h c}{\lambda k_B T}\right) - 1}$$

Spectral flux $(F_\lambda \equiv \frac{dE}{dt\,d\lambda\,dA})$ is the integral of the component of the spectral radiance **in the direction of the observer** $(\cos\theta)$:
$$F_\lambda = \iint B_\lambda \cos\theta \,d\Omega = B_\lambda \int_0^{2\pi} \int_0^{\pi/2} \cos\theta \sin\theta \,d\theta \,d\phi = \pi B_\lambda$$

Spectral luminosity $(L_\lambda \equiv \frac{dE}{dt\,d\lambda})$ is the integral over the surface area of the stellar photosphere:
$$L_\lambda = \iint F_\lambda \,dA = F_\lambda \int_0^{2\pi} \int_{0}^{\pi} R^2 \sin\theta \,d\theta \,d\phi = 4 \pi R^2 F_\lambda$$

Put that all together:
$$L_\lambda = \frac{8 \pi^2 R^2 h c^2}{\lambda^5} \frac{1}{\exp\left(\frac{h c}{\lambda k_B T}\right) - 1}$$


This gives you the spectral luminosity as a function of wavelength $\lambda$ for a star with temperature $T$ and radius $R$. We're going to implement this equation in Python in three different ways:
* using only the built-in math module (this contains the constant $\pi$ and the exponential function)
* using NumPy arrays
* using Astropy quantities

Then calculate spectra for each of the 7 classes of stars (OBAFGKM). In each case, plot the resulting spectra ($L_\lambda$ vs. $\lambda$) to check your work. (Hint: a logarithmic $y$-axis might help.)

In [4]:
import matplotlib.pyplot as plt  # we'll learn more about this next time; for now just use plt.plot(x, y)

## Using only the built-in math module

In [5]:
from math import pi 
# physical constants
R_SUN = 6.96e8  # m
C_LIGHT = 3.00e8  # m / s
H_PLANCK = 6.63e-34  # J / Hz
K_BOLTZMANN = 1.38e-23  # J / K
pi = math.pi

In [24]:
wavelengths =  [400E-9, 440E-9, 480E-9, 520E-9, 560E-9, 600E-9, 640E-9 ] #in meters 
temperatures =  [2100, 3400, 4900, 570, 7200, 9700,30000] #in Celsius
radii = [0.7, 0.8, 1, 1.25, 1.6, 4.2, 6.6]  #in Solar mass 

def temp_to_kelvin(temp):
    lst = []
    for x in temp:
        convert = temp + 273
        lst.append(convert)
def spectral_luminosity(wavelength, temp, radii):
    lst = []
    for x in range(len(wavelength)):
        L_lamda = (8*(pi**2)*(radii[x]**2)*H_PLANCK*(C_LIGHT**2))/(wavelength[x])*(1/(2.71**((H_PLANCK*C_LIGHT)/(wavelength[x]*K_BOLTZMANN*temp[x]))-1))
        lst.append(L_lamda)
    return lst

print(spectral_luminosity(wavelengths, temperatures, radii))
    

    



#print(len(temperatures))
#print(len(radii))

[2.148958599659358e-16, 4.618580066333923e-13, 2.1859345250474974e-11, 1.2501546899595397e-29, 6.280010284138357e-10, 1.2813983165431268e-08, 2.879570149273829e-07]


In [25]:
for 
# do the calculation
L_lamda = (8*(pi**2)*(radii**2)*H_PLANCK*(C_LIGHT**2))/(wavelengths)*(1/(e**((H_PLANCK*C_LIGHT)/(wavelengths*K_BOLTZMANN*temperatures)))-1)
print(L_lamda)


SyntaxError: invalid syntax (3929794588.py, line 1)

In [24]:
# plot the results

## Now with NumPy Arrays

In [ ]:
import numpy as np

In [ ]:
wavelengths =  # complete with numpy array
temperatures =  # complete with numpy array
radii =  # complete with numpy array

In [ ]:
# do the calculation

In [ ]:
# plot the results

## Now with Astropy Quantities

In [ ]:
from astropy import units as u
from astropy import constants as const
from astropy.visualization import quantity_support
quantity_support()  

In [ ]:
wavelengths =  # complete with astropy quantity
temperatures =  # complete with astropy quantity
radii =  # complete with astropy quantity

In [ ]:
# do the calculation

In [ ]:
# plot the results

## Compare the performance of these three methods
Add the `%%timeit` magic command at the top of each calculation cell to see how long it takes to run. Which is fastest? Which is most convenient?

Now increase the number of wavelength samples you are using by a factor of 10. How does this change your timing results?

In reality, which one would you use if you had to do this calculation for 7 stars? Which one would you use if you had to do this calculation for a billion stars (the size of the Gaia catalog)?

(Write your answers here.)

## Save your results using an Astropy Table

In [ ]:
from astropy.table import Table

In [ ]:
table =  # create an astropy table containing your results
table  # look at the results

In [ ]:
# save your table to a file on your hard drive

In [ ]:
table2 =  # read in the file you just saved
table2  # look at the results